### 1 - Importar e criar os métodos

In [14]:
# Importando bibliotecas
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from email_username_password import email, password
import yagmail

In [15]:
# Metodo esperar o aparecimento de elementos na tela
def esperar_elemento(driver : webdriver.Chrome, tempo:int, locate:tuple):
    elemento = WebDriverWait(driver, tempo).until(EC.presence_of_element_located(locate))
    return elemento

# Metodo verificar textos banidos
def is_texto_banido(is_banido:bool, lista_texto:list, nome:str):
    for banido in lista_texto:
        if is_banido:
            break
        if banido.lower() in nome.replace(" ", "").lower():
            is_banido = True
            break
    return is_banido

# Metodo verificar se algum termo da pesquisa não consta no nome
def is_not_contem_texto(is_banido:bool, lista_texto:list, nome:str):
    for termo in lista_texto:
        if is_banido:
            break
        if termo.lower() not in nome.replace(" ", "").lower():
            is_banido = True
            break
    return is_banido

In [16]:
# Método que faz a pesquisa no site do buscapé
def pesquisar_buscape(driver: webdriver.Chrome, nome_produto:str, prod_banidos:str, preco_min: float | int, preco_max : float | int):
    "Pesquisa no site buscapé e retorna um dataframe com o NOME, PREÇO E LINK"
    lista_links = []
    lista_precos = []
    lista_produtos = []
    site = "https://www.buscape.com.br/"

    driver.get(site)
    search = esperar_elemento(driver, 20, (By.CLASS_NAME, "AutoCompleteStyle_input__WAC2Y"))
    search.send_keys(nome_produto,Keys.ENTER)
    links = driver.find_elements(By.CLASS_NAME, "ProductCard_ProductCard_Inner__gapsh")

    for link in links:
        nome = link.find_element(By.CLASS_NAME, "Text_DesktopLabelSAtLarge__wWsED").text
        is_banido = False
        is_banido = is_texto_banido(is_banido, prod_banidos.split(" "), nome)
        is_banido = is_not_contem_texto(is_banido, nome_produto.split(" "), nome)
        
        if not is_banido:
            preco = (esperar_elemento(link, 20, (By.CLASS_NAME, "Text_MobileHeadingS__HEz7L"))).text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            if preco_min <= float(preco) <= preco_max:
                lista_produtos.append(nome)
                lista_precos.append(preco)
                lista_links.append(link.get_attribute("href"))
    tabela_buscape = pd.DataFrame({"Nome":lista_produtos, "Preço":lista_precos, "Links":lista_links})
    return tabela_buscape

In [17]:
# Método que faz a pesquisa no site do google
def pesquisar_google(driver: webdriver.Chrome, nome_produto: str, prod_banidos:str, preco_min:float|int, preco_max:float|int):
    "Pesquisa no googleshop e retorna um dataframe com o NOME, PREÇO E LINK"

    lista_links = []
    lista_precos = []
    lista_produtos = []
    site = "https://shopping.google.com.br/?pli=1"
    
    driver.get(site)
    esperar_elemento(driver, 20, (By.ID, "REsRA")).send_keys(nome_produto, Keys.ENTER)
    esperar_elemento(driver, 20, (By.CLASS_NAME, "baeIxf")).send_keys(str(preco_min), Keys.TAB, str(preco_max), Keys.ENTER)
    
    grid = driver.find_elements(By.CLASS_NAME, "i0X6df")
    
    for elemento in grid:
        nome = elemento.find_element(By.CLASS_NAME, "EI11Pd").text
        preco = elemento.find_element(By.CLASS_NAME, "OFFNJ").text
        div_link = elemento.find_element(By.CLASS_NAME, "mnIHsc")

        is_banido = False
        is_banido = is_texto_banido(is_banido, prod_banidos.split(" "), nome)
        is_banido = is_not_contem_texto(is_banido, nome_produto.split(" "), nome)
        if not is_banido and preco != '':
            lista_produtos.append(nome)
            lista_precos.append(preco)
            lista_links.append(div_link.find_element(By.TAG_NAME, "a").get_attribute("href"))
    tabela_google = pd.DataFrame({"Nome":lista_produtos, "Preço":lista_precos, "Links":lista_links})
    return tabela_google

### 2 - Realizando buscas e salvando os resultados em excel

In [18]:
# Inicializando Navegador
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [19]:
# Lendo planilha com informações
df_pesquisa = pd.read_excel("buscas.xlsx")

# Fazendo buscas
df_google = pd.DataFrame()
df_buscape = pd.DataFrame()

for i in df_pesquisa.index:
    nome_produto, banidos, pre_min, pre_max = df_pesquisa.iloc[i,:].values
    df = pesquisar_google(driver, nome_produto, banidos, pre_min, pre_max)
    df_google = pd.concat([df_google, df], ignore_index=True)
    df = pesquisar_buscape(driver, nome_produto, banidos, pre_min, pre_max)
    df_buscape = pd.concat([df_buscape, df], ignore_index=True)
    
# Encerrando navegador
driver.quit()

In [20]:
# Salvando em excel
name_google = "Pesquisa GoogleSHop.xlsx"
name_buscape = "Pesquisa Buscape.xlsx"
df_google.to_excel(name_google, index=False)
df_buscape.to_excel(name_buscape, index=False)

### 3 - Enviando por e-mail

In [21]:
username = yagmail.SMTP(user=email, password=password)
to = email
subject = "Produtos Encontrados"
email_body = f"""Prezado,

Encontramos produtos dentro da sua faixa de preço:

{df_google[:2].to_html(index=False).replace("  ", "").replace("\n", "")}

{df_buscape[:2].to_html(index=False).replace("  ", "").replace("\n", "")}

O restante está nas planilhas em anexo.

Atenciosamente,
Edson C."""

attachments = [name_google, name_buscape]
username.send(to=to, subject=subject, contents=email_body, attachments=attachments)

{}